In [111]:
import pymongo
import pandas as pd
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
# Dependencies
from bs4 import BeautifulSoup
import requests
db=client.court_db
db.court.drop()
from splinter import Browser
from pprint import pprint

In [120]:
appeal_df = pd.read_csv("appellate.csv") #csv file which you want to import
appeal_df.head()

appeal_df.columns = ['Appellate Case Number', 'Appeal Date Filed', 'Appellant (lost in trial)', 'Appellee (won in trial)', 
                'Case Type', 'COA Case Number', 'COA Case Number_2', 'Trial Court Case Number', 'Trial Court County', 'Trial Court', 'Appellate Court'] 
appeal_df.head()
appeal_df = appeal_df.iloc[:, [0,1,2,3,4,7,8,9,10]]
appeal_df.head()


,Appellate Case Number,Appeal Date Filed,Appellant (lost in trial),Appellee (won in trial),Case Type,Trial Court Case Number,Trial Court County,Trial Court,Appellate Court
0,01-18-01140-CR,12/31/2018,Cecilio Mendoza,The State of Texas,Sexual Assault,83463-CR,Brazoria,149th District Court,COA01
1,01-18-01141-CR,12/31/2018,William Whitfield,The State of Texas,Aggravated Robbery,1586194,Harris,185th District Court,COA01
2,01-18-01144-CR,12/31/2018,"In re Lonnie Henry Rector, Jr.",NaN,Mandamus,714415,Harris,262nd District Court,COA01
3,01-18-01145-CV,12/31/2018,Dwayne Rayshaun Wilson,Empire Towing LLC. and ATC Auto Stora...,Contract,1117336,Harris,Co Civil Ct at Law No 2,COA01
4,01-18-01146-CV,12/31/2018,Thang Bui and Monique Nguyen,Maya Dangelas,Interlocutory,2018-55787,Harris,152nd District Court,COA01


In [122]:
#appeal_df[appeal_df['Trial Court County']== "Harris"]

In [113]:
records = appeal_df.to_dict(orient = 'records')

In [114]:
# Creates a collection in the database and inserts documents
db.appellate.insert_many(records)

In [21]:
###Trial Court Data

In [129]:
#executable_path = {'executable_path':'./chromedriver.exe'}  
#browser = Browser("chrome", **executable_path, headless=False)
#browser = init_browser()
    # URL of page to be scraped
#url = 'https://www.hcdistrictclerk.com/eDocs/Public/Search.aspx?Tab=tabVerdicts'
#browser.visit(url)
#browser.fill('ctl00$ctl00$ctl00$ContentPlaceHolder1$ContentPlaceHolder2$ContentPlaceHolder2$tabSearch$tabVerdicts$txtCaseNumberverdicts', value='100')
#browser.fill('ctl00$ctl00$ctl00$ContentPlaceHolder1$ContentPlaceHolder2$ContentPlaceHolder2$tabSearch$tabVerdicts$txtVerdictStartDate', '1/1/2018')
#browser.fill('ctl00$ctl00$ctl00$ContentPlaceHolder1$ContentPlaceHolder2$ContentPlaceHolder2$tabSearch$tabVerdicts$txtVerdictEndDate', '1/1/2019')
#browser.find_by_name('ctl00$ctl00$ctl00$ContentPlaceHolder1$ContentPlaceHolder2$ContentPlaceHolder2$tabSearch$tabVerdicts$txtVerdictStartDate').type('1/1/2018', slowly=True)



    # Find and click the 'search' button
#button = browser.find_by_name('ctl00$ctl00$ctl00$ContentPlaceHolder1$ContentPlaceHolder2$ContentPlaceHolder2$btnSearch')
    # Interact with elements
#button.click()
    #if browser.is_text_present('splinter.readthedocs.io'):
    #    print("Yes, the official website was found!")
    #else:
    #    print("No, it wasn't found... We need to improve our SEO techniques")
        



In [ ]:
### Civil Judge Data

In [88]:
#executable_path = {'executable_path':'./chromedriver.exe'}  
#browser = Browser("chrome", **executable_path, headless=False)
url = 'https://www.justex.net/Courts/Civil/CivilCourts.aspx'
# Retrieve page with the requests module
response = requests.get(url)
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(response.text, 'html.parser')
#print(soup.prettify())
civil_judges = soup.find_all("a", href=lambda href: href and "CivilCourt" in href)

In [86]:
#tst = civil_judges[0]
#print(tst['href'].split('=')[1])
#tst.text

1


'Judge Kristen Brauchle Hawkins'

In [142]:
civillist = []
i = 0
for x in civil_judges:

        civil_dict = {}
        current_judge = civil_judges[i]
        court_number = (current_judge['href'].split('=')[1])
        judge_name = current_judge.text
        #print(court_number)
        #print(judge_name)
        i += 1
        civil_dict['court_number'] = court_number
        civil_dict['judge_name'] = judge_name
        civillist.append(civil_dict)
#print(civil_dict)
#print(civillist)
civil_df = pd.DataFrame(civillist)
civil_df.head()
civil_df.columns = ['Civil Court Number', 'Civil Judge Name'] 
civil_df.head()
civrecords = civil_df.to_dict(orient = 'records')
db.civiljudges.insert_many(civrecords)

In [ ]:
### Criminal Judge Data

In [139]:
#executable_path = {'executable_path':'./chromedriver.exe'}  
#browser = Browser("chrome", **executable_path, headless=False)
url = 'https://www.justex.net/Courts/Criminal/CriminalCourts.aspx'
# Retrieve page with the requests module
response = requests.get(url)
# Create BeautifulSoup object; parse with 'html.parser'
crimsoup = BeautifulSoup(response.text, 'html.parser')
#print(soup.prettify())
criminal_judges = crimsoup.find_all("a", class_='',href=lambda href: href and "CriminalCourt" in href)
#criminal_judges

In [143]:
crimlist = []
i = 0
for x in criminal_judges:

        crim_dict = {}
        current_crim_judge = criminal_judges[i]
        crim_court_number = (current_crim_judge['href'].split('=')[1])
        crim_judge_name = current_crim_judge.text
        i += 1
        crim_dict['court_number'] = crim_court_number
        crim_dict['judge_name'] = crim_judge_name
        crimlist.append(crim_dict)

#print(crimlist)
criminal_df = pd.DataFrame(crimlist)
criminal_df.head()
criminal_df.columns = ['Criminal Court Number', 'Criminal Judge Name'] 
criminal_df.head()
crimrecords = criminal_df.to_dict(orient = 'records')
db.crimjudges.insert_many(crimrecords)